In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
imsize=(128,128)
ipsize=imsize+(3,)
batch_size=32
data_dir="datasetbkup/train/"
train=tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,seed=123,subset="training",image_size=imsize,batch_size=batch_size,label_mode='categorical')
val=tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,seed=123,subset="validation",image_size=imsize,batch_size=batch_size,label_mode='categorical')
num_classes=len(train.class_names)


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)

# Create a custom model by adding new top layers for your specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change num_classes to match your specific task

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train,validation_data=val, epochs=10)